# Nearest Neighbor Prediction In Equine Time-Series

In this notebook, we use nearest neighbor algorithm to predict the will to win of the next race.

## Intention

We want this sequence predictor to capture the insider's sentiment (He or she may not want to win in a certain race to drive up the odds).

## Hyper-Parameters

* n_lages: the number of time window lags
* n_neighbors: the number of neighbors in k-NN

## Features: X (todo: ask Saleem)

Here, we define the features as the dataset racing starters (e.g. trainer/jockey/owner) concerns

* horse factors
    * last_n_positions
    * last_n_purse
    * average speed
    * winless_in_n
    * distance between tracks
    
* human factors
    * purse
    * 

* final_tote_odds

## Prediction: y (todo: ask Saleem)

We denote the will to win as the following (whether it is in the money):

$$y^{(i)}=True \quad iff \quad {sample}^{(i)}[\mbox{"official_finish_position"}]<=4$$


## Prediction Model: KNN

k-nearest neighbor algorithm is a non-parametric method used for classification and regression. In both cases, the input consists of the k closest training examples in the feature space.

* X: n_samples $\times$ n_features matrix
* y: n_samples binary vector wether 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
import pytest
import sys 
import os
sys.path.append(os.path.abspath("/Users/xiayuxuan/Codes/EquineTrading"))
from horse.betsim.wrap.jcapper import JCapper
from pandas import date_range

In [2]:
n_lags = 2
n_neighbors = 3

In [3]:
# load JCapper Breeders Cup Results file
jcp = JCapper(verbose=True)
jcp.load(date_range(date(2018, 3, 1), date(2018, 3, 7)))
jcp.add_computed_columns()
print('load_jcapper: loaded %d rows' % (len(jcp.df)))

dfp = jcp.df_payout
df = jcp.df

jcapper.load(2018-03-01)
jcapper.load(2018-03-02)
jcapper.load(2018-03-03)
jcapper.load(2018-03-04)
jcapper.load(2018-03-05)
jcapper.load(2018-03-06)
jcapper.load(2018-03-07)
JCapper.load() filtering scratched horses reduced from 4983 to 4521


/Users/xiayuxuan/Codes/EquineTrading/horse/betsim/wrap/jcapper.py:206: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df_payout = concat([df_payout, wps])


load_jcapper: loaded 4521 rows


In [24]:
jcp.df_scratch.to_csv("scratch.csv")

In [17]:
df.to_csv("test.csv")

In [25]:
dfp.to_csv("payout.csv")

In [12]:
df_factors_PILOT = pd.read_csv('../df_factors_PILOT.csv', parse_dates=['date'])

In [14]:
df_factors_PILOT.head()

,race_id,track_id,date,race_number,distance,approx_dist,surface,race_type,sex_restriction,age_restriction,...,x8rank_speed_2of3,x8speed_norm_2of3_par,x8is_longshot,x8is_win_longshot,x8is_exacta_longshot,x8is_trifecta_lonsghot,x8is_superfecta_longshot,x8_outperform,pct_of_purse_earnings,runner_program_number_pp
0,APX_20170703_1,APX,2017-07-03,1,1540.0,False,A,M,N,B,...,3.0,0.614865,0,0.0,0.0,0.0,0.0,-2.0,0.015,7
1,APX_20170703_1,APX,2017-07-03,1,1540.0,False,A,M,N,B,...,5.0,0.486486,1,0.0,0.0,1.0,1.0,1.0,0.100,4
2,APX_20170703_1,APX,2017-07-03,1,1540.0,False,A,M,N,B,...,6.0,0.418919,1,0.0,0.0,0.0,0.0,1.0,0.000,5
3,APX_20170703_1,APX,2017-07-03,1,1540.0,False,A,M,N,B,...,2.0,0.743243,0,0.0,0.0,0.0,0.0,0.0,0.600,2
4,APX_20170703_1,APX,2017-07-03,1,1540.0,False,A,M,N,B,...,1.0,0.770270,0,0.0,0.0,0.0,0.0,0.0,0.250,6


KeyError: 'track_id'

In [3]:
dfX_hist = pd.read_excel("dfX_hist.xlsx")

In [4]:
dfX_hist.head()

,track_id,date,race_id,runner_id,name_runner,jockey_name,index,race_number,distance,approx_dist,...,runner_program_number_pp,race_id_pp,race_race_num_pp,jcp_track_sym_pp,x8_num_runners,x8_num_starters_pp,x8_pct_dirty_HDWPSRRating_par_norm,year,month,weekday
0,APX,2017-07-03,APX_20170703_1,APX_20170703_1_7,CALL ME HANDSOME,SANCHEZ DIEGO,0,1,1540,False,...,7,APX_20170703_1,1,APX,7,7,0,2017,7,0
1,APX,2017-07-03,APX_20170703_1,APX_20170703_1_4,COCOS EDDY,EMIGH CHRISTOPHER A,1,1,1540,False,...,4,APX_20170703_1,1,APX,7,7,0,2017,7,0
2,APX,2017-07-03,APX_20170703_1,APX_20170703_1_5,DA GOOSE,MEZA NICHOLAS,2,1,1540,False,...,5,APX_20170703_1,1,APX,7,7,0,2017,7,0
3,APX,2017-07-03,APX_20170703_1,APX_20170703_1_2,RIO SECO,VALDIVIA JR JOSE,3,1,1540,False,...,2,APX_20170703_1,1,APX,7,7,0,2017,7,0
4,APX,2017-07-03,APX_20170703_1,APX_20170703_1_6,TACO GRANDE,SANJUR SANTO,4,1,1540,False,...,6,APX_20170703_1,1,APX,7,7,0,2017,7,0


## Some indicators

In [40]:
dfX_hist['x8is_favorate_odds'] = dfX_hist.groupby('race_id')['final_tote_odds'].transform(lambda x: x==x.min())
dfX_hist['x8is_favorate_win'] = dfX_hist.transform(lambda x: x["x8is_favorate_odds"]==True and x["official_finish_position"]==1, axis=1)
dfX_hist['x8is_favorate_place'] = dfX_hist.transform(lambda x: x["x8is_favorate_odds"]==True and x["official_finish_position"]==2, axis=1)
dfX_hist['x8is_favorate_show'] = dfX_hist.transform(lambda x: x["x8is_favorate_odds"]==True and x["official_finish_position"]==3, axis=1)
dfX_hist['x8is_favorate_otb'] = dfX_hist.transform(lambda x: x["x8is_favorate_odds"]==True and x["official_finish_position"]>3, axis=1)

In [41]:
last_5_positions = ["finish_pos_{}".format(i) for i in range(5)]
last_10_positions = ["finish_pos_{}".format(i) for i in range(10)]
dfX_hist["x8is_winless_in_5"] = dfX_hist.loc[:,last_5_positions].apply(lambda x: np.sum(x==1), axis=1)
dfX_hist["x8is_winless_in_10"] = dfX_hist.loc[:,last_10_positions].apply(lambda x: np.sum(x==1), axis=1)

In [42]:
dfX_hist.to_csv("dfX_hist_new.csv")
dfX_hist = pd.read_csv("dfX_hist_new.csv")

## Factors

In [ ]:
factors = ['AlternateSpeedPoints','BestHDWSpeedAllWeatherSurface','HDWPSRRating','NumericHDWRunStyleDesignation',
           'SpeedPoints','bestCramerSpeedFigDirt','riderROI','trainerROI','x8runner_HDWPSRRating_norm_par','x8speed_norm_2of3_par',
           'x8ratio_earnings_0_claim']

In [14]:
race_conditions = dfX_hist.loc[:,"race_conditions"]

In [ ]:
race_conditions[race_conditions==""]

In [183]:
def col_past_features(n):
    last_positions = ["finish_pos_{}".format(i) for i in range(n)]
#     last_races = ["race_classification_{}".format(i) for i in range(n)]
    last_races = []
    last_purse = ["purse_{}".format(i) for i in range(n)]
    horse_past_features = last_positions+last_races+last_purse
    return horse_past_features

In [184]:
horse_past_features = col_past_features(n_lags)

In [185]:
factors = ["date","official_finish_position","x8is_pos_top_3","x8is_odds_top_3_favorable","purse","name_runner", "jockey_name", "race_classification","final_tote_odds" ]
factors = factors + horse_past_features

In [186]:
dfX_hist["trainer_name"].unique().shape

(2343,)

In [187]:
n_trainer_data = dfX_hist.groupby("trainer_name").apply(lambda x: len(x))
trainer_names_more_than_25 = n_trainer_data[n_trainer_data>25].index.values

In [188]:
dfX_hist_new = dfX_hist.copy()
dfX_hist_new = dfX_hist_new.set_index(["trainer","race_id"])
dfX_hist_new = dfX_hist_new.sort_index()
# dfX_hist_new = dfX_hist_new.dropna(axis=0)
dfX_hist_new["x8is_pos_top_3"] = dfX_hist_new["official_finish_position"].apply(lambda x: x<=3)
dfX_hist_new["x8is_odds_top_3_favorable"] = dfX_hist_new.groupby("race_id")["final_tote_odds"].transform(lambda x: x.rank()<=3)

In [189]:
idx = pd.IndexSlice
trainer_sequence_df = dfX_hist_new.loc[idx[trainer_names_more_than_25,:],factors].dropna(axis=0)

In [190]:
trainer_sequence_df.head()

date  official_finish_position  \
trainer           race_id                                                
ASMUSSEN STEVEN M BEL_20170703_1  2017-07-03                       4.0   
                  BEL_20170708_11 2017-07-08                       2.0   
                  BEL_20170708_2  2017-07-08                       8.0   
                  BEL_20170709_2  2017-07-09                       7.0   
                  BEL_20170709_7  2017-07-09                       4.0   

                                   x8is_pos_top_3  x8is_odds_top_3_favorable  \
trainer           race_id                                                      
ASMUSSEN STEVEN M BEL_20170703_1            False                      False   
                  BEL_20170708_11            True                      False   
                  BEL_20170708_2            False                       True   
                  BEL_20170709_2            False                       True   
                  BEL_20170709_7            False                      False   

                                    purse       name_runner  \
trainer           race_id                                     
ASMUSSEN STEVEN M BEL_20170703_1    29000         PONTASTIC   
                  BEL_20170708_11   52000    HARLAN'S HUNCH   
                  BEL_20170708_2    67000      WEATHER GIRL   
                  BEL_20170709_2    75000  BREAKIN DAYLIGHT   
                  BEL_20170709_7   150000   GOLDEN MISCHIEF   

                                          jockey_name race_classification  \
trainer           race_id                                                   
ASMUSSEN STEVEN M BEL_20170703_1   SANTANA JR RICARDO            Md 20000   
                  BEL_20170708_11       ORTIZ JR IRAD            Md 75000   
                  BEL_20170708_2    CASTELLANO JAVIER       OClm 40000nw2   
                  BEL_20170709_2         ORTIZ JOSE L            Md Sp Wt   
                  BEL_20170709_7    CASTELLANO JAVIER     Victory Ride-G3   

                                   final_tote_odds  finish_pos_0  \
trainer           race_id                                          
ASMUSSEN STEVEN M BEL_20170703_1              6.10             7   
                  BEL_20170708_11             8.20             3   
                  BEL_20170708_2              3.15             5   
                  BEL_20170709_2              8.90             3   
                  BEL_20170709_7             11.10             2   

                                   finish_pos_1  purse_0  purse_1  
trainer           race_id                                          
ASMUSSEN STEVEN M BEL_20170703_1              4    41000    41000  
                  BEL_20170708_11             8    41000    75000  
                  BEL_20170708_2              4    67000    67000  
                  BEL_20170709_2              3    75000    60000  
                  BEL_20170709_7              4   150000   200000

## Trainer-based Prediction

Here, we conduct our algorithm to each trainer in order to capture the sentiment. 

However, there are some problems need to think over

* The train set is too small
* The features can only be numbers so that distance can be defined (normed vector space). That gives us a problem: how to utilize information like race name. 

In [199]:
for trainer, group in trainer_sequence_df.groupby("trainer"):
#     print(group)
#     print(group)
#     print(group.loc[:,horse_past_features])
    X = group.loc[:,horse_past_features+["final_tote_odds"]].values
#     print(X)
    y = group.loc[:,"x8is_pos_top_3"].values
#     print(y)
    break

In [200]:
print(X)
print(y)

[[  7.00000000e+00   4.00000000e+00   4.10000000e+04   4.10000000e+04
    6.10000000e+00]
 [  3.00000000e+00   8.00000000e+00   4.10000000e+04   7.50000000e+04
    8.20000000e+00]
 [  5.00000000e+00   4.00000000e+00   6.70000000e+04   6.70000000e+04
    3.15000000e+00]
 [  3.00000000e+00   3.00000000e+00   7.50000000e+04   6.00000000e+04
    8.90000000e+00]
 [  2.00000000e+00   4.00000000e+00   1.50000000e+05   2.00000000e+05
    1.11000000e+01]
 [  7.00000000e+00   1.00000000e+00   1.50000000e+05   1.50000000e+05
    1.80000000e+00]
 [  2.00000000e+00   4.00000000e+00   2.31000000e+04   2.10000000e+04
    2.50000000e+00]
 [  1.00000000e+00   2.00000000e+00   3.10000000e+04   3.10000000e+04
    3.06000000e+01]
 [  5.00000000e+00   2.00000000e+00   1.50000000e+05   8.20000000e+04
    2.70000000e+00]
 [  1.00000000e+00   5.00000000e+00   2.80000000e+04   3.45000000e+04
    5.00000000e-01]
 [  4.00000000e+00   2.00000000e+00   5.33000000e+04   7.00000000e+04
    3.00000000e+00]
 [  8.0000

In [202]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=n_neighbors)
neigh.fit(X[:-2], y[:-2]) 
KNeighborsClassifier(...)
print(neigh.predict(X[-2:]))

[False  True]


In [203]:
y[-2:]

array([ True,  True], dtype=bool)